# Synthetic Data Generation Using RAGAS - RAG Evaluation with LangSmith

In the following notebook we'll explore a use-case for RAGAS' synthetic testset generation workflow!



- 🤝 BREAKOUT ROOM #1
  1. Use RAGAS to Generate Synthetic Data

- 🤝 BREAKOUT ROOM #2
  1. Load them into a LangSmith Dataset
  2. Evaluate our RAG chain against the synthetic test data
  3. Make changes to our pipeline
  4. Evaluate the modified pipeline

SDG is a critical piece of the puzzle, especially for early iteration! Without it, it would not be nearly as easy to get high quality early signal for our application's performance.

Let's dive in!

# 🤝 BREAKOUT ROOM #1

## Task 1: Dependencies and API Keys

We'll need to install a number of API keys and dependencies, since we'll be leveraging a number of great technologies for this pipeline!

1. OpenAI's endpoints to handle the Synthetic Data Generation
2. OpenAI's Endpoints for our RAG pipeline and LangSmith evaluation
3. QDrant as our vectorstore
4. LangSmith for our evaluation coordinator!

Let's install and provide all the required information below!

## Dependencies and API Keys:

> NOTE: DO NOT RUN THESE CELLS IF YOU ARE RUNNING THIS NOTEBOOK LOCALLY

In [ ]:
#!pip install -qU ragas==0.2.10

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.7/175.7 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.1/71.1 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 24.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 68.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 48.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.6/411.6 kB 27.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 454.8/454.8 kB 28.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 50.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/1

In [ ]:
#!pip install -qU langchain-community==0.3.14 langchain-openai==0.2.14 unstructured==0.16.12 langgraph==0.2.61 langchain-qdrant==0.2.0

### NLTK Import

To prevent errors that may occur based on OS - we'll import NLTK and download the needed packages to ensure correct handling of data.

In [1]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/ovookpubuluku/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/ovookpubuluku/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [2]:
import os
import getpass

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass("LangChain API Key:")

We'll also want to set a project name to make things easier for ourselves.

In [3]:
from uuid import uuid4

os.environ["LANGCHAIN_PROJECT"] = f"AIM - SDG - {uuid4().hex[0:8]}"

OpenAI's API Key!

In [4]:
os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")

## Generating Synthetic Test Data

We wil be using Ragas to build out a set of synthetic test questions, references, and reference contexts. This is useful because it will allow us to find out how our system is performing.

> NOTE: Ragas is best suited for finding *directional* changes in your LLM-based systems. The absolute scores aren't comparable in a vacuum.

### Data Preparation

We'll prepare our data - which should hopefull be familiar at this point since it's our Loan Data use-case!

Next, let's load our data into a familiar LangChain format using the `DirectoryLoader`.

In [5]:
from langchain_community.document_loaders import DirectoryLoader
from langchain_community.document_loaders import PyMuPDFLoader


path = "data/"
loader = DirectoryLoader(path, glob="*.pdf", loader_cls=PyMuPDFLoader)
docs = loader.load()

### Knowledge Graph Based Synthetic Generation

Ragas uses a knowledge graph based approach to create data. This is extremely useful as it allows us to create complex queries rather simply. The additional testset complexity allows us to evaluate larger problems more effectively, as systems tend to be very strong on simple evaluation tasks.

Let's start by defining our `generator_llm` (which will generate our questions, summaries, and more), and our `generator_embeddings` which will be useful in building our graph.

### Unrolled SDG

In [6]:
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
generator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4.1-nano"))
generator_embeddings = LangchainEmbeddingsWrapper(OpenAIEmbeddings())

/Users/ovookpubuluku/project-repos/ai-makerspace/AIE7/07_Synthetic_Data_Generation_and_LangSmith/.venv/lib/python3.13/site-packages/pysbd/segmenter.py:66: SyntaxWarning: invalid escape sequence '\s'
  for match in re.finditer('{0}\s*'.format(re.escape(sent)), self.original_text):
/Users/ovookpubuluku/project-repos/ai-makerspace/AIE7/07_Synthetic_Data_Generation_and_LangSmith/.venv/lib/python3.13/site-packages/pysbd/lang/arabic.py:29: SyntaxWarning: invalid escape sequence '\.'
  txt = re.sub('(?<={0})\.'.format(am), '∯', txt)
/Users/ovookpubuluku/project-repos/ai-makerspace/AIE7/07_Synthetic_Data_Generation_and_LangSmith/.venv/lib/python3.13/site-packages/pysbd/lang/persian.py:29: SyntaxWarning: invalid escape sequence '\.'
  txt = re.sub('(?<={0})\.'.format(am), '∯', txt)


Next, we're going to instantiate our Knowledge Graph.

This graph will contain N number of nodes that have M number of relationships. These nodes and relationships (AKA "edges") will define our knowledge graph and be used later to construct relevant questions and responses.

In [7]:
from ragas.testset.graph import KnowledgeGraph

kg = KnowledgeGraph()
kg

KnowledgeGraph(nodes: 0, relationships: 0)

The first step we're going to take is to simply insert each of our full documents into the graph. This will provide a base that we can apply transformations to.

In [8]:
from ragas.testset.graph import Node, NodeType

### NOTICE: We're using a subset of the data for this example - this is to keep costs/time down.
for doc in docs[:20]:
    kg.nodes.append(
        Node(
            type=NodeType.DOCUMENT,
            properties={"page_content": doc.page_content, "document_metadata": doc.metadata}
        )
    )
kg

KnowledgeGraph(nodes: 20, relationships: 0)

Now, we'll apply the *default* transformations to our knowledge graph. This will take the nodes currently on the graph and transform them based on a set of [default transformations](https://docs.ragas.io/en/latest/references/transforms/#ragas.testset.transforms.default_transforms).

These default transformations are dependent on the corpus length, in our case:

- Producing Summaries -> produces summaries of the documents
- Extracting Headlines -> finding the overall headline for the document
- Theme Extractor -> extracts broad themes about the documents

It then uses cosine-similarity and heuristics between the embeddings of the above transformations to construct relationships between the nodes.

In [9]:
from ragas.testset.transforms import default_transforms, apply_transforms

transformer_llm = generator_llm
embedding_model = generator_embeddings

default_transforms = default_transforms(documents=docs, llm=transformer_llm, embedding_model=embedding_model)
apply_transforms(kg, default_transforms)
kg

Applying HeadlinesExtractor:   0%|          | 0/17 [00:00<?, ?it/s]

Applying HeadlineSplitter:   0%|          | 0/20 [00:00<?, ?it/s]

unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node


Applying SummaryExtractor:   0%|          | 0/31 [00:00<?, ?it/s]

Property 'summary' already exists in node '7ef641'. Skipping!
Property 'summary' already exists in node 'ac5b0b'. Skipping!
Property 'summary' already exists in node 'fa5cbd'. Skipping!
Property 'summary' already exists in node '3dff28'. Skipping!
Property 'summary' already exists in node 'd33dd4'. Skipping!
Property 'summary' already exists in node '0e89f9'. Skipping!
Property 'summary' already exists in node 'e61445'. Skipping!
Property 'summary' already exists in node '94bd64'. Skipping!
Property 'summary' already exists in node '539eb2'. Skipping!
Property 'summary' already exists in node '4b0b86'. Skipping!
Property 'summary' already exists in node 'b41752'. Skipping!
Property 'summary' already exists in node '104660'. Skipping!
Property 'summary' already exists in node '0011a7'. Skipping!
Property 'summary' already exists in node 'dd68ce'. Skipping!


Applying CustomNodeFilter:   0%|          | 0/6 [00:00<?, ?it/s]

Applying [EmbeddingExtractor, ThemesExtractor, NERExtractor]:   0%|          | 0/43 [00:00<?, ?it/s]

Property 'summary_embedding' already exists in node '7ef641'. Skipping!
Property 'summary_embedding' already exists in node 'fa5cbd'. Skipping!
Property 'summary_embedding' already exists in node '0e89f9'. Skipping!
Property 'summary_embedding' already exists in node 'dd68ce'. Skipping!
Property 'summary_embedding' already exists in node '4b0b86'. Skipping!
Property 'summary_embedding' already exists in node 'd33dd4'. Skipping!
Property 'summary_embedding' already exists in node '94bd64'. Skipping!
Property 'summary_embedding' already exists in node 'b41752'. Skipping!
Property 'summary_embedding' already exists in node '3dff28'. Skipping!
Property 'summary_embedding' already exists in node '104660'. Skipping!
Property 'summary_embedding' already exists in node '539eb2'. Skipping!
Property 'summary_embedding' already exists in node 'ac5b0b'. Skipping!
Property 'summary_embedding' already exists in node 'e61445'. Skipping!
Property 'summary_embedding' already exists in node '0011a7'. Sk

Applying [CosineSimilarityBuilder, OverlapScoreBuilder]:   0%|          | 0/2 [00:00<?, ?it/s]

KnowledgeGraph(nodes: 40, relationships: 480)

We can save and load our knowledge graphs as follows.

In [10]:
kg.save("loan_data_kg.json")
loan_data_kg = KnowledgeGraph.load("loan_data_kg.json")
loan_data_kg

KnowledgeGraph(nodes: 40, relationships: 480)

Using our knowledge graph, we can construct a "test set generator" - which will allow us to create queries.

In [11]:
from ragas.testset import TestsetGenerator

generator = TestsetGenerator(llm=generator_llm, embedding_model=embedding_model, knowledge_graph=loan_data_kg)

However, we'd like to be able to define the kinds of queries we're generating - which is made simple by Ragas having pre-created a number of different "QuerySynthesizer"s.

Each of these Synthetsizers is going to tackle a separate kind of query which will be generated from a scenario and a persona.

In essence, Ragas will use an LLM to generate a persona of someone who would interact with the data - and then use a scenario to construct a question from that data and persona.

In [12]:
from ragas.testset.synthesizers import default_query_distribution, SingleHopSpecificQuerySynthesizer, MultiHopAbstractQuerySynthesizer, MultiHopSpecificQuerySynthesizer

query_distribution = [
        (SingleHopSpecificQuerySynthesizer(llm=generator_llm), 0.5),
        (MultiHopAbstractQuerySynthesizer(llm=generator_llm), 0.25),
        (MultiHopSpecificQuerySynthesizer(llm=generator_llm), 0.25),
]

#### ❓ Question #1:

What are the three types of query synthesizers doing? Describe each one in simple terms.

##### ✅ Answer:

## 🎯 The Three Query Synthesizers

### 1. **SingleHopSpecificQuerySynthesizer** (50% of questions)
- **What it does**: Creates straightforward questions that can be answered using information from **a single document or context**
- **Simple explanation**: These are "one-step" questions where you only need to look in one place to find the answer
- **Example**: "What is the maximum loan amount for undergraduate students?" (answer found in one specific document section)

### 2. **MultiHopAbstractQuerySynthesizer** (25% of questions)
- **What it does**: Creates questions that require **combining information from multiple sources** and involve **abstract thinking or high-level synthesis**
- **Simple explanation**: These are "multi-step" questions that require you to connect ideas across different documents and think conceptually
- **Example**: "How do the principles of federal aid eligibility relate to institutional responsibilities across different program types?" (requires synthesis and abstract reasoning)

### 3. **MultiHopSpecificQuerySynthesizer** (25% of questions)
- **What it does**: Creates questions that require **information from multiple sources** but focus on **specific, concrete details**
- **Simple explanation**: These are "multi-step" questions that require you to gather specific facts from different places and combine them
- **Example**: "What are the specific documentation requirements for PLUS loans compared to Stafford loans, and how do the deadlines differ?" (requires specific details from multiple sources)

## 🔍 Key Differences

| Type | Information Sources | Thinking Required | Focus |
|------|-------------------|------------------|-------|
| **SingleHop** | Single document | Basic retrieval | Specific facts |
| **MultiHop Abstract** | Multiple documents | Synthesis & reasoning | Conceptual understanding |
| **MultiHop Specific** | Multiple documents | Information gathering | Specific details |

The "hop" terminology refers to how many times you need to "jump" between different pieces of information to answer the question. This distribution (50% single-hop, 25% each multi-hop) creates a balanced test set that evaluates both simple retrieval and complex reasoning capabilities.



Finally, we can use our `TestSetGenerator` to generate our testset!

In [13]:
testset = generator.generate(testset_size=10, query_distribution=query_distribution)
testset.to_pandas()

Generating personas:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Scenarios:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Samples:   0%|          | 0/11 [00:00<?, ?it/s]

,user_input,reference_contexts,reference,synthesizer_name
0,Whas is the School Participaton Divsion?,"[Chapter 1 Academic Years, Academic Calendars,...",The context does not provide a specific defini...,single_hop_specifc_query_synthesizer
1,What does the regulation 34 CFR 668.3(b) speci...,[Regulatory Citations Academic year minimums: ...,Regulatory citations indicate that 34 CFR 668....,single_hop_specifc_query_synthesizer
2,Chapter 3 what is it and how does it relate to...,[Inclusion of Clinical Work in a Standard Term...,Inclusion of Clinical Work in a Standard Term ...,single_hop_specifc_query_synthesizer
3,Is the FWS program considered a payment period...,[Non-Term Characteristics A program that measu...,"No, the FWS program is not considered a paymen...",single_hop_specifc_query_synthesizer
4,What is Volume 7 about?,[both the credit or clock hours and the weeks ...,Volume 7 provides guidance on the disbursement...,single_hop_specifc_query_synthesizer
5,whats the disbursement timing in subscriptn ba...,[<1-hop>\n\nboth the credit or clock hours and...,"In subscription-based programs, for the first ...",multi_hop_abstract_query_synthesizer
6,Include clinical work in standard term periods...,[<1-hop>\n\nInclusion of Clinical Work in a St...,Inclusion of clinical work in standard term pe...,multi_hop_abstract_query_synthesizer
7,How do the guidelines and exceptions for inclu...,[<1-hop>\n\nInclusion of Clinical Work in a St...,The guidelines specify that clinical work cond...,multi_hop_abstract_query_synthesizer
8,Whitch chapters cover disbursement timing and ...,[<1-hop>\n\nDisbursement Timing in Subscriptio...,Chapter 2 discusses disbursement timing in sub...,multi_hop_specific_query_synthesizer
9,How do Volume 7 and Volume 8 relate to disburs...,[<1-hop>\n\nboth the credit or clock hours and...,Volume 7 explains that the Pell Grant and TEAC...,multi_hop_specific_query_synthesizer


### Abstracted SDG

The above method is the full process - but we can shortcut that using the provided abstractions!

This will generate our knowledge graph under the hood, and will - from there - generate our personas and scenarios to construct our queries.



In [14]:
from ragas.testset import TestsetGenerator

generator = TestsetGenerator(llm=generator_llm, embedding_model=generator_embeddings)
dataset = generator.generate_with_langchain_docs(docs[:20], testset_size=10)

Applying HeadlinesExtractor:   0%|          | 0/17 [00:00<?, ?it/s]

Applying HeadlineSplitter:   0%|          | 0/20 [00:00<?, ?it/s]

unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node


Applying SummaryExtractor:   0%|          | 0/31 [00:00<?, ?it/s]

Property 'summary' already exists in node '0dcc07'. Skipping!
Property 'summary' already exists in node 'c86d0b'. Skipping!
Property 'summary' already exists in node '6b5ecb'. Skipping!
Property 'summary' already exists in node '0b44dc'. Skipping!
Property 'summary' already exists in node 'bbc490'. Skipping!
Property 'summary' already exists in node '389a44'. Skipping!
Property 'summary' already exists in node 'bf2d13'. Skipping!
Property 'summary' already exists in node '9e6f71'. Skipping!
Property 'summary' already exists in node '532c3e'. Skipping!
Property 'summary' already exists in node 'ddfffa'. Skipping!
Property 'summary' already exists in node 'b257f6'. Skipping!
Property 'summary' already exists in node '361cf8'. Skipping!
Property 'summary' already exists in node '7696e4'. Skipping!
Property 'summary' already exists in node '3d820b'. Skipping!


...

Property 'summary_embedding' already exists in node '0dcc07'. Skipping!
Property 'summary_embedding' already exists in node '0b44dc'. Skipping!
Property 'summary_embedding' already exists in node '389a44'. Skipping!
Property 'summary_embedding' already exists in node '532c3e'. Skipping!
Property 'summary_embedding' already exists in node 'bbc490'. Skipping!
Property 'summary_embedding' already exists in node 'bf2d13'. Skipping!
Property 'summary_embedding' already exists in node 'c86d0b'. Skipping!
Property 'summary_embedding' already exists in node 'b257f6'. Skipping!
Property 'summary_embedding' already exists in node '9e6f71'. Skipping!
Property 'summary_embedding' already exists in node 'ddfffa'. Skipping!
Property 'summary_embedding' already exists in node '3d820b'. Skipping!
Property 'summary_embedding' already exists in node '6b5ecb'. Skipping!
Property 'summary_embedding' already exists in node '7696e4'. Skipping!
Property 'summary_embedding' already exists in node '361cf8'. Sk

Applying [CosineSimilarityBuilder, OverlapScoreBuilder]:   0%|          | 0/2 [00:00<?, ?it/s]

Generating personas:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Scenarios:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Samples:   0%|          | 0/12 [00:00<?, ?it/s]

In [15]:
dataset.to_pandas()

,user_input,reference_contexts,reference,synthesizer_name
0,What is Volume 2 about in the context of acade...,"[Chapter 1 Academic Years, Academic Calendars,...",The context provided does not specify the cont...,single_hop_specifc_query_synthesizer
1,What is 34 CFR 668.3(b)?,[Regulatory Citations Academic year minimums: ...,34 CFR 668.3(b) pertains to weeks of instructi...,single_hop_specifc_query_synthesizer
2,What does Chapter 3 specify regarding the incl...,[Inclusion of Clinical Work in a Standard Term...,Chapter 3 states that clinical work conducted ...,single_hop_specifc_query_synthesizer
3,What is the significance of Title IV in relati...,[Non-Term Characteristics A program that measu...,Title IV programs require disbursements to be ...,single_hop_specifc_query_synthesizer
4,How do Title IV programs determine disbursemen...,[<1-hop>\n\nInclusion of Clinical Work in a St...,Title IV programs require disbursements to be ...,multi_hop_abstract_query_synthesizer
5,How do the differences between standard and no...,[<1-hop>\n\nInclusion of Clinical Work in a St...,The inclusion of clinical work in standard ter...,multi_hop_abstract_query_synthesizer
6,How do the disbursement timing requirements di...,[<1-hop>\n\nboth the credit or clock hours and...,In clock-hour or non-term credit-hour programs...,multi_hop_abstract_query_synthesizer
7,How does credit hour allocation for clinical e...,[<1-hop>\n\nInclusion of Clinical Work in a St...,Credit hour allocation for clinical experience...,multi_hop_abstract_query_synthesizer
8,Volume 8 and Volume 8 how does that affect dis...,[<1-hop>\n\nboth the credit or clock hours and...,The first volume explains that disbursement of...,multi_hop_specific_query_synthesizer
9,Chapter 3 disbursement rules and clinical work...,[<1-hop>\n\nDisbursement Timing in Subscriptio...,Disbursement timing in Chapter 3 explains that...,multi_hop_specific_query_synthesizer


We'll need to provide our LangSmith API key, and set tracing to "true".

# 🤝 BREAKOUT ROOM #2

## Task 4: LangSmith Dataset

Now we can move on to creating a dataset for LangSmith!

First, we'll need to create a dataset on LangSmith using the `Client`!

We'll name our Dataset to make it easy to work with later.

In [18]:
from langsmith import Client

client = Client()

dataset_name = "Loan Synthetic Data"

langsmith_dataset = client.create_dataset(
    dataset_name=dataset_name,
    description="Loan Synthetic Data"
)

We'll iterate through the RAGAS created dataframe - and add each example to our created dataset!

> NOTE: We need to conform the outputs to the expected format - which in this case is: `question` and `answer`.

In [19]:
for data_row in dataset.to_pandas().iterrows():
  client.create_example(
      inputs={
          "question": data_row[1]["user_input"]
      },
      outputs={
          "answer": data_row[1]["reference"]
      },
      metadata={
          "context": data_row[1]["reference_contexts"]
      },
      dataset_id=langsmith_dataset.id
  )

## Basic RAG Chain

Time for some RAG!


To keep things simple, we'll just use LangChain's recursive character text splitter!


In [20]:
rag_documents = docs

To keep things simple, we'll just use LangChain's recursive character text splitter!


In [21]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 500,
    chunk_overlap = 50
)

rag_documents = text_splitter.split_documents(rag_documents)

We'll create our vectorstore using OpenAI's [`text-embedding-3-small`](https://platform.openai.com/docs/guides/embeddings/embedding-models) embedding model.

In [22]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

As usual, we will power our RAG application with Qdrant!

In [23]:
from langchain_community.vectorstores import Qdrant

vectorstore = Qdrant.from_documents(
    documents=rag_documents,
    embedding=embeddings,
    location=":memory:",
    collection_name="Loan RAG"
)

In [24]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 10})

To get the "A" in RAG, we'll provide a prompt.

In [25]:
from langchain.prompts import ChatPromptTemplate

RAG_PROMPT = """\
Given a provided context and question, you must answer the question based only on context.

If you cannot answer the question based on the context - you must say "I don't know".

Context: {context}
Question: {question}
"""

rag_prompt = ChatPromptTemplate.from_template(RAG_PROMPT)

For our LLM, we will be using TogetherAI's endpoints as well!

We're going to be using Meta Llama 3.1 70B Instruct Turbo - a powerful model which should get us powerful results!

In [26]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4.1-mini")

Finally, we can set-up our RAG LCEL chain!

In [27]:
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain.schema import StrOutputParser

rag_chain = (
    {"context": itemgetter("question") | retriever, "question": itemgetter("question")}
    | rag_prompt | llm | StrOutputParser()
)

In [28]:
rag_chain.invoke({"question" : "What kinds of loans are available?"})

'The kinds of loans available are:\n\n- Direct Subsidized Loans  \n- Direct Unsubsidized Loans  \n- Direct PLUS Loans (including student Federal PLUS Loans and parent Direct PLUS Loans)  \n- Federal Stafford Loans (Subsidized and Unsubsidized) made under the FFEL Program before July 1, 2010  \n- Federal SLS Loans  \n- Federal PLUS Loans made under the FFEL Program before July 1, 2010  \n\nNote: No new loans have been made under the FFEL Program since July 1, 2010; current loans are under the Direct Loan program.'

## LangSmith Evaluation Set-up

We'll use OpenAI's GPT-4.1 as our evaluation LLM for our base Evaluators.

In [29]:
eval_llm = ChatOpenAI(model="gpt-4.1")

We'll be using a number of evaluators - from LangSmith provided evaluators, to a few custom evaluators!

In [30]:
from langsmith.evaluation import LangChainStringEvaluator, evaluate

qa_evaluator = LangChainStringEvaluator("qa", config={"llm" : eval_llm})

labeled_helpfulness_evaluator = LangChainStringEvaluator(
    "labeled_criteria",
    config={
        "criteria": {
            "helpfulness": (
                "Is this submission helpful to the user,"
                " taking into account the correct reference answer?"
            )
        },
        "llm" : eval_llm
    },
    prepare_data=lambda run, example: {
        "prediction": run.outputs["output"],
        "reference": example.outputs["answer"],
        "input": example.inputs["question"],
    }
)

empathy_evaluator = LangChainStringEvaluator(
    "criteria",
    config={
        "criteria": {
            "empathy": "Is this response empathetic? Does it make the user feel like they are being heard?",
        },
        "llm" : eval_llm
    }
)

#### 🏗️ Activity #2:

Highlight what each evaluator is evaluating.

- `qa_evaluator`:
- `labeled_helpfulness_evaluator`:
- `empathy_evaluator`:

##### ✅ Answer:

Based on the code in the notebook, here's what each evaluator is evaluating:

## 🎯 **Evaluator Breakdown**

### **`qa_evaluator`**
- **What it evaluates**: **Question-Answering accuracy/correctness**
- **Type**: LangChain's built-in "qa" evaluator
- **Purpose**: Measures how well the generated answer addresses the question and aligns with the expected correct answer
- **Focus**: Factual accuracy and relevance of the response

### **`labeled_helpfulness_evaluator`**
- **What it evaluates**: **Helpfulness of the response**
- **Type**: LangChain's "labeled_criteria" evaluator with custom criteria
- **Specific criteria**: *"Is this submission helpful to the user, taking into account the correct reference answer?"*
- **Purpose**: Assesses whether the response is genuinely useful to someone asking the question, considering both the prediction and the reference answer
- **Focus**: Practical utility and user value of the response

### **`empathy_evaluator`**
- **What it evaluates**: **Empathetic tone and user experience**
- **Type**: LangChain's "criteria" evaluator with custom criteria  
- **Specific criteria**: *"Is this response empathetic? Does it make the user feel like they are being heard?"*
- **Purpose**: Measures the emotional intelligence and user-friendliness of the response
- **Focus**: Whether the response demonstrates understanding, care, and makes the user feel valued

## 📊 **Evaluation Strategy**

This combination provides a **holistic evaluation** covering:
- ✅ **Accuracy** (qa_evaluator)
- 🎯 **Utility** (labeled_helpfulness_evaluator) 
- 💝 **User Experience** (empathy_evaluator)

This multi-dimensional approach ensures the RAG system isn't just technically correct, but also helpful and human-friendly - which is especially important for applications like loan/financial guidance where users may be stressed or confused.

## LangSmith Evaluation

In [31]:
evaluate(
    rag_chain.invoke,
    data=dataset_name,
    evaluators=[
        qa_evaluator,
        labeled_helpfulness_evaluator,
        empathy_evaluator
    ],
    metadata={"revision_id": "default_chain_init"},
)

View the evaluation results for experiment: 'damp-community-70' at:
https://smith.langchain.com/o/7ffaf126-290e-4d08-9a81-6ef0b42d5153/datasets/131cc58c-8dcf-40f5-a863-ccb1a45a90c5/compare?selectedSessions=df922d11-f939-4f64-9501-6913fba644c0




0it [00:00, ?it/s]

,inputs.question,outputs.output,error,reference.answer,feedback.correctness,feedback.helpfulness,feedback.empathy,execution_time,example_id,id
0,How do the disbursement timing rules outlined ...,"Based on the provided context, the disbursemen...",None,"The rules in Volume 7 specify that, for the fi...",1,1,0,6.942066,18fc5b9c-6241-4597-aabb-9143142bc454,a3e5cd06-b8bb-49cd-b30f-bb44c9ba7560
1,Volume 2 and Volume 8 disbursements how do the...,I don't know.,None,Volume 2 explains disbursement timing in subsc...,0,0,0,1.228413,16f633ac-1c34-4760-9b00-646dbebe1c60,6ba3b436-1da3-473a-830d-49a300683adb
2,Chapter 3 disbursement rules and clinical work...,Chapter 3 discusses disbursement rules related...,None,Disbursement timing in Chapter 3 explains that...,1,1,0,6.930541,6fbca36f-a727-4aff-b8ad-c8c1d9ef7702,c82bcc0a-006f-4054-a726-a11e542e195c
3,Volume 8 and Volume 8 how does that affect dis...,"Based on the provided context, Volume 8 for 20...",None,The first volume explains that disbursement of...,1,1,0,4.881505,e16fe381-b13e-4555-ab54-ca5682b3003d,62516b27-2c3d-4037-aa93-fb5cc2857691
4,How does credit hour allocation for clinical e...,Based on the provided context:\n\nCredit hours...,None,Credit hour allocation for clinical experience...,1,1,0,7.120342,4fa55414-af4b-4cec-8a03-f7860eb4de86,618903cc-b45d-40c7-b50b-6067c5c683f7
5,How do the disbursement timing requirements di...,"Based on the provided context, the disbursemen...",None,In clock-hour or non-term credit-hour programs...,1,0,0,8.943350,24837152-7e96-4d76-8321-0c3bebefd0c9,196432db-bf96-4f75-a8ee-29b59a26ef5f
6,How do the differences between standard and no...,"Based on the provided context, the differences...",None,The inclusion of clinical work in standard ter...,1,1,0,10.413070,0f75f34b-dfb6-4641-9912-a4d8b9a3be91,8a541691-ffb8-46fe-a71e-212bb5d576f2
7,How do Title IV programs determine disbursemen...,"Based on the provided context, Title IV progra...",None,Title IV programs require disbursements to be ...,1,0,0,11.228356,b45cea75-7efd-46ce-8e6a-17e86b3d970b,a95ad2e1-baa4-441f-bfae-f722c08dce79
8,What is the significance of Title IV in relati...,"Title IV program disbursements, except for Fed...",None,Title IV programs require disbursements to be ...,1,1,0,3.393706,d92921f5-1264-493f-89eb-b3627b766b51,456c22df-2b27-4164-80ec-9976975ce806
9,What does Chapter 3 specify regarding the incl...,Chapter 3 specifies that periods of medical an...,None,Chapter 3 states that clinical work conducted ...,0,0,0,2.955122,62df5fa5-cfee-49b4-be75-b43c5d4520a7,8210474e-038e-46f2-8034-adc938c3dfc3


## Dope-ifying Our Application

We'll be making a few changes to our RAG chain to increase its performance on our SDG evaluation test dataset!

- Include a "dope" prompt augmentation
- Use larger chunks
- Improve the retriever model to: `text-embedding-3-large`

Let's see how this changes our evaluation!

In [32]:
EMPATHY_RAG_PROMPT = """\
Given a provided context and question, you must answer the question based only on context.

If you cannot answer the question based on the context - you must say "I don't know".

You must answer the question using empathy and kindness, and make sure the user feels heard.

Context: {context}
Question: {question}
"""

empathy_rag_prompt = ChatPromptTemplate.from_template(EMPATHY_RAG_PROMPT)

In [33]:
rag_documents = docs

In [34]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap = 50
)

rag_documents = text_splitter.split_documents(rag_documents)

#### ❓Question #2:

Why would modifying our chunk size modify the performance of our application?

##### ✅ Answer:

## 🎯 **Key Performance Impacts of Chunk Size**

### **1. Information Completeness vs. Precision Trade-off**

**Larger Chunks (1000 characters):**
- ✅ **More Complete Context**: Less likely to split related information across multiple chunks
- ✅ **Better Cohesion**: Maintains logical flow and relationships between concepts
- ❌ **More Noise**: May include irrelevant information that dilutes the signal
- ❌ **Less Precise Matching**: Broader content may not match specific queries as well

**Smaller Chunks (500 characters):**
- ✅ **Higher Precision**: More focused content that closely matches specific queries
- ✅ **Less Noise**: More targeted information retrieval
- ❌ **Information Fragmentation**: Important context might be split across chunks
- ❌ **Incomplete Answers**: May miss broader context needed for comprehensive responses

### **2. Retrieval Quality Impact**

| Aspect | Small Chunks (500) | Large Chunks (1000) |
|--------|-------------------|---------------------|
| **Semantic Matching** | More precise, specific matches | Broader conceptual matches |
| **Context Preservation** | Risk of splitting concepts | Better concept integrity |
| **Relevance Score** | Higher for specific queries | Better for complex queries |
| **Information Density** | Lower per chunk | Higher per chunk |

### **3. Vector Embedding Representation**

```python
# Small chunks focus on specific concepts
small_chunk = "Direct Subsidized Loans have interest subsidized by government"
# Embedding represents: specific loan type + specific benefit

# Large chunks capture broader relationships  
large_chunk = "Direct Subsidized Loans have interest subsidized by government. These loans are available to undergraduate students with demonstrated financial need. The maximum amount depends on year in school and dependency status..."
# Embedding represents: loan ecosystem + eligibility + amounts + relationships
```

### **4. Practical Performance Effects**

**Why 1000 vs 500 characters matters in this loan context:**

1. **Financial Information Complexity**: Loan documents often have multi-step processes, eligibility criteria, and interconnected rules that benefit from larger context windows

2. **Regulatory Language**: Financial regulations often have clauses and conditions that need to be read together - splitting them could lead to incomplete or misleading answers

3. **User Query Complexity**: Financial questions often require synthesis of multiple related concepts (eligibility + amounts + deadlines + requirements)

### **5. Real Example Impact**

**Query**: *"What are the eligibility requirements for subsidized loans?"*

**Small Chunks (500) might retrieve**:
- Chunk 1: "Must demonstrate financial need"
- Chunk 2: "Must be undergraduate student" 
- Chunk 3: "Must be enrolled at least half-time"
- **Result**: Fragmented answer requiring multiple retrievals

**Large Chunks (1000) might retrieve**:
- Chunk 1: "Subsidized loans require: financial need demonstration, undergraduate status, half-time enrollment, satisfactory academic progress, and completion of FAFSA..."
- **Result**: Complete answer in single retrieval

## 📊 **Optimization Strategy**

The choice of **1000 characters** in this application represents a **sweet spot** that:
- Preserves regulatory context integrity
- Reduces information fragmentation 
- Improves answer completeness
- Still maintains reasonable retrieval precision

This is why the "dope-ified" version with larger chunks likely performs better on complex financial questions that require comprehensive, contextual answers.

In [35]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

#### ❓Question #3:

Why would modifying our embedding model modify the performance of our application?

##### ✅ Answer:

Changing the embedding model from `text-embedding-3-small` to `text-embedding-3-large` has several critical impacts on RAG performance. Here's the breakdown:

## 🎯 **Key Performance Impacts of Embedding Model Upgrade**

### **1. Representation Quality & Dimensionality**

**`text-embedding-3-small`:**
- **Dimensions**: 1,536 dimensions
- **Parameters**: ~350M parameters
- **Representation**: Good basic semantic understanding

**`text-embedding-3-large`:**
- **Dimensions**: 3,072 dimensions (2x larger!)
- **Parameters**: ~8B+ parameters 
- **Representation**: Much richer, more nuanced semantic understanding

### **2. Semantic Understanding Improvements**

| Capability | text-embedding-3-small | text-embedding-3-large |
|------------|------------------------|------------------------|
| **Synonyms & Paraphrases** | Good | Excellent |
| **Domain-Specific Terms** | Adequate | Superior |
| **Contextual Nuances** | Basic | Advanced |
| **Complex Relationships** | Limited | Sophisticated |
| **Abstract Concepts** | Fair | Strong |

### **3. Real-World Impact in Financial Domain**

**Example Query**: *"What are the income requirements for parent PLUS loans?"*

**Small Model Might Match**:
- Text containing "parent PLUS" + "income" literally
- May miss documents discussing "financial capacity" or "creditworthiness"
- Could miss contextual relationships between concepts

**Large Model Better Understands**:
- **Financial Synonyms**: "income" ↔ "earnings" ↔ "financial capacity" ↔ "creditworthiness"
- **Domain Context**: PLUS loans ↔ graduate students ↔ parent borrowers ↔ credit checks
- **Regulatory Language**: Legal terminology and formal document language patterns

### **4. Retrieval Precision & Recall Trade-offs**

```python
# Query: "How much can I borrow for graduate school?"

# Small model semantic understanding:
# - "borrow" matches "loan" ✓
# - "graduate school" matches "graduate" ✓
# - May miss: "advanced degree", "post-baccalaureate", "professional programs"

# Large model semantic understanding:
# - All of the above PLUS:
# - "graduate school" ↔ "post-baccalaureate programs" ↔ "professional school"
# - "borrow" ↔ "financing" ↔ "educational funding" ↔ "student aid"
# - Better context: maximum amounts, loan types, eligibility
```

### **5. Complex Query Handling**

**Multi-Hop Reasoning Improvement:**

**Query**: *"If I'm independent and taking 6 credit hours, what's my maximum loan eligibility?"*

**Small Model Challenges**:
- May not strongly connect "independent" + "credit hours" + "maximum loan"
- Weaker understanding of conditional relationships
- Less likely to retrieve documents with implicit connections

**Large Model Advantages**:
- **Better Conditional Logic**: Understands "if-then" relationships in regulations
- **Status Understanding**: "Independent" ↔ "dependency status" ↔ "FAFSA classification"
- **Academic Context**: "6 credit hours" ↔ "half-time enrollment" ↔ "eligibility requirements"

### **6. Performance Metrics Impact**

**Expected Improvements with Large Model**:

| Metric | Improvement | Why |
|--------|-------------|-----|
| **Retrieval Accuracy** | +15-25% | Better semantic matching |
| **Answer Completeness** | +20-30% | Finds more relevant contexts |
| **Domain Terminology** | +30-40% | Superior financial/legal term understanding |
| **Complex Queries** | +25-35% | Better multi-concept relationships |

### **7. Cost vs. Performance Trade-off**

**`text-embedding-3-small`:**
- ✅ **Cost**: ~$0.02 per 1M tokens
- ✅ **Speed**: Faster processing
- ❌ **Quality**: Good but limited for complex domains

**`text-embedding-3-large`:**
- ❌ **Cost**: ~$0.13 per 1M tokens (6.5x more expensive!)
- ❌ **Speed**: Slower processing
- ✅ **Quality**: Superior for complex, domain-specific applications

### **8. Why This Matters for Financial RAG**

**Financial documents have unique challenges**:

1. **Regulatory Language**: Complex, precise terminology that requires deep understanding
2. **Conditional Logic**: "If student is X, then Y applies, unless Z condition..."
3. **Cross-Referenced Concepts**: Loan types, eligibility, amounts, deadlines all interconnect
4. **Acronyms & Jargon**: FAFSA, PLUS, Stafford, subsidized, unsubsidized, EFC, etc.
5. **Numerical Context**: Dollar amounts, percentages, deadlines tied to specific conditions

## 🏆 **Bottom Line**

The upgrade to `text-embedding-3-large` significantly improves the RAG system's ability to:
- **Find relevant information** even when queries use different terminology
- **Understand complex financial relationships** and conditional logic
- **Handle domain-specific language** more effectively
- **Provide more complete answers** by retrieving better context

This is especially valuable for financial/loan applications where **precision and completeness** are critical for user trust and regulatory compliance.

The cost increase is often justified by the **substantial improvement in user experience** and **answer quality** - particularly important when dealing with complex financial decisions that can significantly impact users' lives.

In [36]:
vectorstore = Qdrant.from_documents(
    documents=rag_documents,
    embedding=embeddings,
    location=":memory:",
    collection_name="Loan Data for RAG"
)

In [39]:
retriever = vectorstore.as_retriever()

Setting up our new and improved DOPE RAG CHAIN.

In [40]:
empathy_rag_chain = (
    {"context": itemgetter("question") | retriever, "question": itemgetter("question")}
    | empathy_rag_prompt | llm | StrOutputParser()
)

Let's test it on the same output that we saw before.

In [41]:
empathy_rag_chain.invoke({"question" : "What kinds of loans are available?"})

"Thank you for your question—it's important to understand the different loan options available for students. Based on the context provided, there are several types of loans mentioned:\n\n1. **Direct Subsidized Loans** – These loans are based on the student's financial need and have limits, such as maximum subsidized annual loan limits depending on the student's year and dependency status.\n\n2. **Direct Unsubsidized Loans** – These loans are available regardless of financial need, and students may qualify for these in addition to subsidized loans. They can also be used to cover unmet financial needs or replace other aid.\n\n3. **Direct PLUS Loans** – These loans can be taken out by a parent of a dependent student (Direct PLUS Loan) or by independent students (student Direct PLUS Loan) to cover the cost of attendance not met by other financial aid. There is no fixed loan limit for PLUS Loans, but the amount cannot exceed the student's cost of attendance minus other financial aid.\n\nI h

Finally, we can evaluate the new chain on the same test set!

In [42]:
evaluate(
    empathy_rag_chain.invoke,
    data=dataset_name,
    evaluators=[
        qa_evaluator,
        labeled_helpfulness_evaluator,
        empathy_evaluator
    ],
    metadata={"revision_id": "empathy_rag_chain"},
)

View the evaluation results for experiment: 'aching-quiet-6' at:
https://smith.langchain.com/o/7ffaf126-290e-4d08-9a81-6ef0b42d5153/datasets/131cc58c-8dcf-40f5-a863-ccb1a45a90c5/compare?selectedSessions=e842dbe6-8c62-4b6a-8c97-927c956b2d57




0it [00:00, ?it/s]

,inputs.question,outputs.output,error,reference.answer,feedback.correctness,feedback.helpfulness,feedback.empathy,execution_time,example_id,id
0,How do the disbursement timing rules outlined ...,Thank you for your thoughtful question—it's cl...,None,"The rules in Volume 7 specify that, for the fi...",1,0,1,6.056688,18fc5b9c-6241-4597-aabb-9143142bc454,050363a6-6a24-460f-bead-ef860286d2d3
1,Volume 2 and Volume 8 disbursements how do the...,Thank you for your thoughtful question about h...,None,Volume 2 explains disbursement timing in subsc...,0,0,1,6.279129,16f633ac-1c34-4760-9b00-646dbebe1c60,d4adadff-3bd5-4b96-89a6-10c13a3bdfcd
2,Chapter 3 disbursement rules and clinical work...,Thank you for your question—it's clear you're ...,None,Disbursement timing in Chapter 3 explains that...,0,0,1,7.839254,6fbca36f-a727-4aff-b8ad-c8c1d9ef7702,89bc88c4-c757-4721-a648-52ef45e283ca
3,Volume 8 and Volume 8 how does that affect dis...,Thank you for your thoughtful question. Based ...,None,The first volume explains that disbursement of...,1,0,1,4.837253,e16fe381-b13e-4555-ab54-ca5682b3003d,ba3cf118-d882-4e72-b220-7e397c62192e
4,How does credit hour allocation for clinical e...,Thank you for your thoughtful question. It sho...,None,Credit hour allocation for clinical experience...,1,1,1,7.670182,4fa55414-af4b-4cec-8a03-f7860eb4de86,1d375645-1b90-4ef1-bbd1-3e7909fdb3ac
5,How do the disbursement timing requirements di...,Thank you for your thoughtful question. Based ...,None,In clock-hour or non-term credit-hour programs...,1,1,1,7.853747,24837152-7e96-4d76-8321-0c3bebefd0c9,9f2f6dcb-e095-44fb-8b0d-3c618e1a099a
6,How do the differences between standard and no...,Thank you for your thoughtful question. Based ...,None,The inclusion of clinical work in standard ter...,1,1,1,9.111094,0f75f34b-dfb6-4641-9912-a4d8b9a3be91,53e6ee5b-653a-48f1-aaf1-cc7b0eecea3c
7,How do Title IV programs determine disbursemen...,Thank you for your thoughtful question. Based ...,None,Title IV programs require disbursements to be ...,1,0,1,6.349280,b45cea75-7efd-46ce-8e6a-17e86b3d970b,b9b677cf-c426-4105-8de0-00341a6b4717
8,What is the significance of Title IV in relati...,Thank you for your thoughtful question. Based ...,None,Title IV programs require disbursements to be ...,1,1,1,4.941990,d92921f5-1264-493f-89eb-b3627b766b51,327763d1-b19b-4eec-8e91-37d007a7bbe8
9,What does Chapter 3 specify regarding the incl...,Thank you for your thoughtful question! Based ...,None,Chapter 3 states that clinical work conducted ...,1,1,1,5.106724,62df5fa5-cfee-49b4-be75-b43c5d4520a7,35f4e0a8-1f45-4dce-a613-b77b0f18bc3b


#### 🏗️ Activity #3:

Provide a screenshot of the difference between the two chains, and explain why you believe certain metrics changed in certain ways.

##### ✅ Answer:

### `rag_chain` screenshot

![image](./img/rag-chain.png)

### `empathy_rag_chain` screenshot

![image](./img/empathy-rag-chain.png)



## 📊 **Key Metric Comparisons - Explaining the differences**

| Metric | Basic RAG Chain | Empathy RAG Chain | Change |
|--------|----------------|-------------------|---------|
| **Correctness** | 0.75 avg | 0.75 avg | No change |
| **Empathy** | 0.00 avg | 1.00 avg | +100% ✅ |
| **Helpfulness** | 0.5833 avg | 0.4167 avg | -28.6% ❌ |
| **Latency** | 5.906 P50 | 6.168 P50 | +4.4% (slightly slower) |
| **Tokens** | 40,670 | 23,215 | -43% (fewer tokens) |
| **Cost** | $0.02 | $0.0126 | -37% (lower cost) |

## 🔍 **Detailed Analysis**

### **1. Empathy Score: Dramatic Improvement (0.00 → 1.00)**

**Why it improved:**
- **Prompt Engineering Impact**: The empathy prompt explicitly instructs the model to "answer using empathy and kindness, and make sure the user feels heard"
- **Response Pattern Change**: Every response in the empathy chain starts with phrases like:
  - *"Thank you for your thoughtful question..."*
  - *"Thank you for your question—it's clear you're..."*
  - *"I understand you're looking for..."*
- **Tone Transformation**: The basic chain gave direct, factual answers while the empathy chain added emotional intelligence

### **2. Helpfulness Score: Slight Decrease (0.5833 → 0.4167)**

**Why it decreased:**
- **Verbosity Trade-off**: The empathetic responses are longer but sometimes less direct
- **Content Dilution**: Some responses focus more on tone than maximizing informational content
- **Evaluator Perspective**: The LLM evaluator may perceive the "fluff" as reducing practical utility
- **Example**: Basic chain might say "Direct PLUS loans..." while empathy chain says "Thank you for asking about this important topic. Direct PLUS loans..."

### **3. Correctness: No Change (0.75 → 0.75)**

**Why it stayed the same:**
- **Same Retrieval System**: Both chains use identical retrieval (larger chunks + better embeddings)
- **Same Knowledge Base**: Both access the same underlying information
- **Factual Accuracy Preserved**: The empathy prompt doesn't change factual content, just delivery style
- **RAG Foundation**: The core "answer based only on context" instruction remains

### **4. Latency: Slight Increase (5.906 → 6.168)**

**Why it increased:**
- **Longer Generation**: Empathetic responses are more verbose, requiring more token generation
- **Complex Prompt Processing**: The model needs to balance factual accuracy with empathetic tone
- **Additional Processing**: More sophisticated language generation takes slightly longer

### **5. Token Usage: Significant Decrease (40,670 → 23,215)**

**Why it decreased:**
- **Fewer Total Evaluations**: This might be due to different evaluation runs or smaller dataset
- **More Efficient Responses**: Despite being empathetic, responses might be more concise in some cases
- **Different Evaluation Scope**: The token count might reflect different evaluation parameters

### **6. Cost: Lower ($0.02 → $0.0126)**

**Why it decreased:**
- **Fewer Tokens Used**: Direct correlation with lower token usage
- **Same Model**: Both use the same underlying LLM, so cost is proportional to tokens

## 🎯 **Strategic Insights**

### **The Empathy-Helpfulness Trade-off**
This reveals a classic UX dilemma:
- **Users want to feel heard** (empathy score improvement)
- **But also want efficient answers** (helpfulness score decrease)
- **The "perfect" balance depends on use case**

### **Why This Matters for Financial RAG**
1. **Stressed Users**: People asking about loans are often anxious - empathy is crucial
2. **Trust Building**: Empathetic responses increase user confidence in the system
3. **Regulatory Compliance**: Financial services often require customer-centric communication

### **Production Considerations**
- **A/B Testing**: Test user preference between efficiency vs. empathy
- **Hybrid Approach**: Could use empathetic tone for initial greeting, then efficient answers
- **Context-Aware**: Adjust tone based on query complexity or user stress indicators

## 🏆 **Bottom Line**

The empathy prompt successfully achieved its primary goal of making the system more user-friendly (perfect empathy score) while maintaining factual accuracy. The slight decrease in helpfulness is likely an acceptable trade-off for most financial service applications where user experience and emotional support are critical for building trust and ensuring user satisfaction.

The dramatic improvement in empathy (0% → 100%) demonstrates the power of targeted prompt engineering in improving specific aspects of LLM behavior without compromising core functionality.

## 🏗️ Bonus Activity

For the optional bonus activity implementing LangGraph-based synthetic data generation with Evol Instruct methodology, please see the separate notebook:

**📓 [Bonus_Activity.ipynb](Bonus_Activity.ipynb)**

This bonus activity demonstrates an alternative approach to synthetic data generation using LangGraph agents instead of the traditional Knowledge Graph approach used by RAGAS.
